### <u> Generate the Planetary Orbit sonifications used in the "_Audible Universe_" planetarium show </u>

**First, import relevant modules:**

In [ ]:
%reload_ext autoreload 
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import ffmpeg as ff
import wavio as wav
from strauss.sonification import Sonification
from strauss.sources import SingleObject
from strauss import channels
from strauss.score import Score
import numpy as np
from strauss.generator import Sampler
import IPython.display as ipd
import os

**Collate the notes we are using to represent each planet and their orbital periods, as well as the length of each sonification**

Then, combine these into dictionaries so they can be easily indexed

In [ ]:
# planet names
planets = ['Mercury',
          'Venus',
          'Earth',
          'Mars',
          'Jupiter',
          'Saturn',
          'Uranus',
          'Neptune']

# notes representing each planet
notes = [['Gb4', 'Db5', 'E5','Gb4']]

# orbital period of each planet in days
periods = np.array([88,
                     224.7,
                     365.2,
                     687,
                     4331,
                     10747,
                     30589,
                     59800])

# sonification lengths for each planet
lengths = [126]*4 + [84]*4

# put these into dictionaries
chorddict = dict(zip(planets,notes))
lendict = dict(zip(planets, lengths))
perioddict = dict(zip(planets,periods*(lendict['Neptune']/(0.75 * periods[-1]))))

**Specify the audio system to use** _(use `'stereo'` by default but for the planetarium `'5.1'` is used)_

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

**Now, set-up the sampler and the mapping functions and limits of mapped quantities**

In [ ]:
# set up sampler
sampler = Sampler("../data/samples/solar_system")

# we want to loop the orchestral samples
sampler.modify_preset({'looping':'forward', # looping style
                       'loop_start': 7.0, # start of loop in seconds
                       'loop_end': 9.4}) # end of loop in seconds

# mapping functions and their limits
mapvals =  {'phi': lambda x : x,
            'theta': lambda x : x,
            'pitch' : lambda x: x,
            'volume' : lambda x : x,
           'time_evo' : lambda x : x}

maplims =  {'phi': (0, 360),
            'theta': (0, 180),
            'pitch' : (0, 1),
            'volume' : (0, 1),
           'time_evo' : (0,1)}

**Specify which planet you want to sonify:**

In [ ]:
# modify these for each planet
planet = "Mars"
panphase = 0

**Render sonification for specified planet...**

In [ ]:
# volume swell is directly ahead
volphase = panphase + 90

# setup score
score =  Score(chorddict[planet], lendict[planet])

# data dict
n = 10000
orbits_per_sonification = lendict[planet]/perioddict[planet]
orbital_phi = (np.linspace(0,orbits_per_sonification,n)%1)*360
data = {'phi':orbital_phi + panphase,
        'theta':np.ones(n)*90., # constant theta of 90 deg
        'pitch':np.ones(n),     # constant pitch
        'volume':np.sin((orbital_phi + volphase) * np.pi/180.)*0.4+0.6,
        'time_evo': np.linspace(0,1,n)
        }

# set up source
events = SingleObject(mapvals.keys())
events.fromdict(data)
events.apply_mapping_functions(mapvals, maplims)

soni = Sonification(score, events, sampler, system)
soni.render()

**Listen to and plot the waveforms from the sonification:**

In [ ]:
# I should adapt this to an inbuilt function of the sonification class at some point...
time = soni.out_channels['0'].samples / soni.out_channels['0'].samprate
for i in range(len(soni.out_channels)):
    plt.plot(time[::20], soni.out_channels[str(i)].values[::20]-1.2*i, label=soni.channels.labels[i])

plt.xlabel('Time (s)')
plt.ylabel('Relative Amplitude')
plt.legend(frameon=False, loc=5)
plt.xlim(-time[-1]*0.05,time[-1]*1.2)
for s in plt.gca().spines.values():
    s.set_visible(False)
plt.gca().get_yaxis().set_visible(False)

if len(soni.channels.labels) == 1:
    # we have used 441000 Hz everywhere above as standard, but to quickly hear the sonification sped up / slowed down,
    # you can modify the 'rate' argument below (e.g. multiply by 0.5 for half speed, by 2 for double speed, etc)
    outfmt = np.column_stack([soni.out_channels['0'].values, soni.out_channels['0'].values]).T
else:
    outfmt = np.column_stack([soni.out_channels['0'].values, soni.out_channels['1'].values]).T
ipd.Audio(outfmt,rate=soni.out_channels['0'].samprate, autoplay=False)

**Combine and save sonification to a multi-channel wav** 

NOTE: Change `"../../FILENAME.wav"` to your filepath of choice. By default, the sound file is normalised to that of the highest amplitude sample, but can be set to a lower normalisation by setting the `master_volume` parameter to a value between `0.` and `1.`.

In [ ]:
soni.save_combined("../../FILENAME.wav", True, master_volume=1.0)

In [ ]:
data = {'phi':[],
        'theta':[], # constant theta of 90 deg
        'pitch':[],     # constant pitch
        'volume':[],
        'time_evo': []}

n = 10000

for k in planets:
    panphase = np.random.random()*360.
    # volume swell is directly ahead
    volphase = panphase + 90

    # setup score
    score =  Score(chorddict[planet], lendict[planet])

    # data dict
    orbits_per_sonification = lendict[planet]/perioddict[planet]
    orbital_phi = (np.linspace(0,orbits_per_sonification,n)%1)*360

    data['phi'].append(orbital_phi + panphase)
    data['theta'].append(np.ones(n)*90.) 
    data['pitch'].append(np.ones(n))
    data['volume'].append(np.sin((orbital_phi + volphase) * np.pi/180.)*0.4+0.6)
    data['time_evo'].append(np.linspace(0,1,n))

# set up source
events = SingleObject(mapvals.keys())
events.fromdict(data)
events.apply_mapping_functions(mapvals, maplims)

soni = Sonification(score, events, sampler, system)
soni.render()